# 0) Instructions:
Please complete the workbook below. Some of the calculations are already ready to be "run". However, please read the text carefully to find questions that you should answer for credit. Remember that to answer a question in text, you click "insert", then "insert cell below", switch the input from "code" to "markdown", type your answer, and finally click the run button to set your text in stone. In a few questions, you will need to do some calculations on your own. Hint: for these calculations you can copy, paste, and modify code that is above the calculation that you need to do. You may work by yourself or in groups of 2. Please remember to put your name on top, remember to save the workbook, and remember to upload to Canvas.

Please run the code below to load the *tidyverse* set of packages, load the *rpart* package and change the default image size to 5 x 4. Please ignore the output.

In [ ]:
library(tidyverse)
library(rpart)
options(repr.plot.width = 5, repr.plot.height = 4)

# 1) Non-parametric alternative to ANOVA
In this part of lab, we will use the Kruskal-Wallis test in order to determine if there are differences in the distribution of a continous random variable between 3 or more populations. 

In the below article, the researchers wanted to determine (amoung other aims), if the percentage of cells in G2 phase was associated with the severity of prostate cancer tumors. We will answer this research question today.

O. Nativ, Y. Raz, H.Z. Winkler, Y. Hosaka, E.T. Boyle, T.M. Therneau, G.M. Farrow, R.P. Meyers, H. Zincke, and M.M Lieber.  (1988). Prognostic value of flow cytometric nuclear DNA analysis in stage C prostate carcinoma. *Surgical Forum, 39*, 685-687.

First, we will load the data and do some processing steps (which you can ignore). Basically these steps consist of converting some variables stored as numeric to categorical variables (which R calls "factors"), combining some factor levels, and removing missing data:

In [ ]:
data(stagec)
stagec <- stagec[!is.na(stagec$g2) & !is.na(stagec$gleason),]
stagec$grade <- as.factor(stagec$grade)
stagec$gleason <- as.character(stagec$gleason)
stagec$gleason[stagec$gleason %in% c("3", "4")] <- "3-4"
stagec$gleason[stagec$gleason %in% c("8", "9", "10")] <- "8-10"
stagec$gleason <- as.factor(stagec$gleason)

Okay let's use the "head" function to show us the first few observations from the dataset:

In [ ]:
head(stagec)

For us, we will be interested in the "g2" variable (the % of cells counted that are in G2 phase) and the "gleason" variable. This variable is a scoring system for judging the appearance of cells in the prostate gland. We will want to determine if the distribution of the "g2" variable differs between different levels of the "gleason" variable. Our first step will be to determine if the data looks approximately normally distributed. Let's make a Q-Q plot now!

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 5.5)
qqnorm(stagec$g2)
qqline(stagec$g2)

**Question for you.** Based on the Q-Q plot above, do you believe that the "g2" random variable is approximately normally distributed? Please answer below in a new markdown cell.

Let's also take a look at a histogram of the same data:

In [ ]:
ggplot(stagec, aes(x = g2, y = ..density..)) + geom_histogram(color = "black", fill = "grey70", bins = 22) + theme_bw()

**Question for you.** Based on the histogram above, how would you describe the distribution of the "g2" random variable? Is there skew present? If so on which side? Is the distribution symmetric? Is it unimodal, bimodal, or multi-modal? Please answer below in a new markdown cell.

Okay, before we set up our hypotheses and statistical test, let's examine how many observations (indidual tumors) there are by the Gelason type. We can use the "table" function to do this. This tabulates the number of times a specific value occurs in a variable: 

In [ ]:
table(stagec$gleason)

**Question for you**. Based on the graphical distribution of the "g2" variable you have seen thus far, do you think it is more appropriate to use parametric tests for comparing population means (like ANOVA), or non-parametric tests for comparing population distributions to answer our research question (whether the percentage of cells in G2 phase differs by Gleason category)? Please answer this question in a new markdown cell below. 

**Question for you**. Given your answer from above, please state the null and alternative hypotheses for the test that we should conduct. Rather than using fancy Greek letters, you can just write "Delta" (for example). Please write these hypotheses below in a new markdown cell. 

Let's take a final step in our exploratory analysis and produce a boxplot showing the distribution of the "g2" variable by "gleason": 

In [ ]:
options(repr.plot.width = 6, repr.plot.height = 5)
ggplot(stagec, aes(x = gleason, group = gleason, y = g2, fill = gleason)) + geom_boxplot() + theme_bw()

**Question for you**. Based on the boxplot above, do you think that we will find evidence that not all of the distributions are the same? This question only requires you to guess. Please answer below with your guess in a new markdown cell. 

## 1.1) Doing the test "by hand": 
Okay, so to compute a test statistic for our Kruskal-Wallis rank sum test, we will need to combine all of the "g2" values from each group, assign ranks to each value, and then sum the ranks within each group. First let's take a look at our full set of values from the "g2" variable. To do this we will use the "\\$" notation, which retrieves a variable from a "data.frame" (in our case it will retrieve the "g2" variable from the data.frame "stagec"): 

In [ ]:
stagec$g2

So now, to rank the values of this "g2" variable, we will need to determine their order. We can use the "order" function to do this. The output of this function can be a bit tricky to understand. Basically, this function will return the order in the original data that would arrange the data from smallest to largest. So below, you will find the first value is "52". This means that the smallest "g2" value comes from the 52 row in the dataset. Let's take a look below: 

In [ ]:
order(stagec$g2)

Let's now save this ordering as "newOrder":

In [ ]:
newOrder <- order(stagec$g2)

Now to make use of the ordering, we will want to order our "g2" variable using this list. To do this we will use the '[]'. Let's take a quick look at how this is used. To get the first value of the "g2" variable (which is not yet ordered) we would do this: 

In [ ]:
stagec$g2[1]

Or to get the fisrt through the 10th values we would do this:

In [ ]:
stagec$g2[1:10]

Now to order the "g2" variable we will want to follow the same process but using our "newOrder" list:

In [ ]:
stagec$g2[newOrder]

Looking at the above, you can see that we have ordered the "g2" variable from smallest to largest. However, we will later need to know not only the order of the "g2" values, but also which group they come from. So instead of just ordering the "g2" variable, we should order the entire dataset. To do this we will do the following: 

In [ ]:
stagec[newOrder,]

Okay, so above we can see how to order the data.frame by the ordering of the "g2" variable. Now let's permanantly order the observations in the data.frame; we will overwrite the "stagec" data.frame, with an ordered version of itself: 

In [ ]:
stagec <- stagec[order(stagec$g2),]

Alright! Now we are ready to assign the ranks. We can do this using the "rank" function. The rank function has an argument that allows us to specify how ties are handled. We can specify 'ties.method = "average"' so that ties are handled by assigning the average rank to all of the tied observations in one tie group. 

In [ ]:
rank(stagec$g2, ties.method = "average")

**Question for you**. Based on the ranks above, are there any ties present? Please answer below.

We can also check for ties using the "table" function. The idea here will be to count each time we see a specific value. Under the specific value, you will see how many times it was observed:

In [ ]:
table(stagec$g2)

**Question for you**. Based on the above table, how many groups of tied observations are there? Is it more than 6? Please answer below. 

Okay, let's see what we have done in creating this rank variable "G2Rank" by printing out the first few observations:

In [ ]:
head(stagec)

Okay, since there are not too many ties, we will just compute our test statistic $H$ directly. For this we need to determine the sum of ranks $T_j$ for each of the groups $j$. Below we will subset the data.frame for the observations where the Gleason category is "3-4". The "==" is a test for equivalence. So if we say 'stagec$gleason == "3-4"', we are asking R to test each observation and see if the Gleason category is "3-4", if yes the test value will be "TRUE". We then subset the data.frame for cases where this value is true: 

In [ ]:
stagec[stagec$gleason == "3-4",]

Okay, so above we see the subset of the data.frame where "gleason" is "3-4". However, we don't want the whole data.frame, we just want the "G2Rank" values for finding our $T_j$ for this group. So we can ask for just those values: 

In [ ]:
stagec$G2Rank[stagec$gleason == "3-4"]

Now for $T_j$ for this group we need to sum them:

In [ ]:
sum(stagec$G2Rank[stagec$gleason == "3-4"])

**Question for you.** Now that you have seen the process, please compute $T_j$ for the Gleason category "5". You should be able to copy, paste, and modify code from above into new code cells below. Once you have the rank sum $T_j$, please type it in a markdown cell below all of the code cells you created. 

Continuing on, we need to find $T_j^2 / n_j$ for each of the groups, as this will go into our test statistic $H$. To see the $n_j$ for each of the groups we can use the "table" function again: 

In [ ]:
table(stagec$gleason)

Okay, so now, let's compute $T_j^2 / n_j$ for the group where the Gleason category is "3-4":

In [ ]:
319^2 / 8

Okay, so this has all been painful so far. However, we will use a shortcut to get the rest of the $T_j^2 / n_j$. This uses the pipping (%>%) provided by a package called *dplyr*. Basically this is a trick where we pipe things through one function to another:

In [ ]:
stagec %>% group_by(gleason) %>% summarize(sumOfRanks = sum(G2Rank), n = n(), sumOfRanksSqDivn = sumOfRanks^2 / n)

Okay, so now we will save (as "sr") that table that we just made, as we will need it later: 

In [ ]:
sr <- stagec %>% group_by(gleason) %>% summarize(sumOfRanks = sum(G2Rank), n = n(), sumOfRanksSqDivn = sumOfRanks^2 / n)

Okay, it is time to find our test statistic $H$. Here is the formula... $H = \frac{12}{N (N+1)} \sum_{j=1}^p \frac{T_j^2}{n_j} - 3(N+1)$

Okay, from our table above we determined the sample size from each group $n_j$. Let's add these all together to give us $N$: 

In [ ]:
sum(sr$n)

Okay, now to find $\sum_{j=1}^p \frac{T_j^2}{n_j}$, we can use that same "sr" table, but now we want to sum the variable we called "sumOfRanksSqDivn": 

In [ ]:
sum(sr$sumOfRanksSqDivn)

Now we can find $H$: 

In [ ]:
12 / (136 * 137) * sum(sr$sumOfRanksSqDivn) - 3 * (136 + 1)

So we now have a test statistic.

**Question for you.** What is the sampling distribution of this test statistic if the null hypothesis is true. If you don't remember, please revisit the lecture slides. Please answer below in a new markdown cell. 

**Question for you.** Now that we remember the sampling distribution of the test statistic, let's find the critical value for a test assuming $\alpha = 0.05$. For this you can use the book tables, or use on of the functions: "qf" for quantile from an $F$ distribution, "qt" for quantiles from a $t$ distribution, "qnorm" for quantiles from a normal distribution, or "qchisq" for quantiles from a $\chi^2$ distribution. Please find this quantile below: 

**Question for you**. Now please find the p-value of this test. It is probably easiest to use one of the following functions: "pf", "pt", "pnorm", "pchisq". You will want to think about whether you want a p-value for the probability less than or equal to your test statistic ('lower.tail = TRUE'), or a p-value for the proability greater than or equal to your test statistic ('lower.tail = FALSE'):

If you forget how to use a function you can just type ?function. Try this below: 

In [ ]:
?pf

## 1.2) By software
As you might expect, it is time to do all of the work we just did in one step.

We will use the "kruskal.test" function. This function can use something called formula notation "Dependent variable ~ Independent variable" as the first argument: 

In [ ]:
kruskal.test(g2 ~ gleason, data = stagec)

### 1.2.1) Pairwise comparisons
From above we can see that there is evidence that the distribution of the "g2" variable differs by "gleason". Now, if we want to determine pairs of populations differ from eachother, we can use the "pairwise.wilcoxon.test" function. This function has a few arguments; first we specify the dependent variable, second we specify the indpendent variable (the variable that says the population; and finally we can adjust the p-values to account for multiple comparisons (remember Mr. Bonferroni? He's back to save you from Type I errors). First, let's make the pairwise comparisons with no p-value adjustment:

In [ ]:
pairwise.wilcox.test(stagec$g2, stagec$gleason, p.adjust.method = "none")

Now we will ask for the pairwise comparisons with the "Bonferroni" corrected p-values:

In [ ]:
pairwise.wilcox.test(stagec$g2, stagec$gleason, p.adjust.method = "bonf")

**Question for you**. Based on the above table of pairwise comparisons, which of the populations have different distributions? 

**Caveat** In this section we analyzed the data assuming that each Gleason score, that is 3-4, 5, 6, 7, 8-10, was a separate population. We did not treat this Gleason variable as a being orderd, but rather treated it as a nominal variable. In this case, this probably a horrible idea. We did it to illustrate the Kruskal test. It is a horrible idea because we are not recognizing that there is some order amoung the Gleason score categories. That is 3-4 < 5 < 6 < 7 < 8-10. There are other superior methods for ordinal data out there. 

# 2) Inferences regarding a single population proportion
Now, we will pivot to something very different--proportions. In this part we will determine point estimates and confidence intervals for a population proportion as well as conduct a hypothesis test for one. First, we will define some data. This data comes from the article: Sallan SE, Cronin C, Zelen M, Zinberg NE (1980), "Antiemetics in patients receiving chemotherapy for cancer: a randomized comparison of delta-9-tetrahydrocannabinol and prochlorperazine," *New England Journal of Medicine, 302*(3) p.135-138. This article evaluated the efficacy of both THC and Prochlorperaizne for the treatment of vomiting and nausea in patients undergoing chemotherapy. Below we will create the data as a matrix or table: 

In [5]:
antiemetics <- matrix(c(36, 43, 16, 62), nrow = 2, ncol = 2, byrow = TRUE)
colnames(antiemetics) <- c("Effective", "NotEffective")
rownames(antiemetics) <- c("THC", "Prochlorperazine")

This data is in a $2\times 2$ data that shows the count of study subjects in each category. Let's take a look at the data below: 

In [6]:
antiemetics

,Effective,NotEffective
THC,36,43
Prochlorperazine,16,62


## 2.1) Point estimates and confidence intervals
First, let's determine a point estimate for the propotion of chemotherapy patients for whom THC would be effective as an antiemetic. We will first save this sample proportion as "piHatTHC", and then we will view it in the cell after we save it. Remember that to compute a sample proportion we do $\hat{\pi}=y/n$:

In [ ]:
piHatTHC <- 36 / (36 + 43)

In [ ]:
piHatTHC

Now let's suppose that we would like to determine a 95% confidence interval for the population proportion of chemotherapy patients for whom THC would be effective as an antiemetic. We will determine both a Wald CI and a WAC CI. For the Wald CI remember that we need to use the following form: $\hat{\pi} \pm z_{\alpha / 2} \text{SE}(\hat{\pi})$. To determine $\text{SE}(\hat{\pi})$ we will use $\text{SE}(\hat{\pi}) = \sqrt{\frac{(\hat{\pi})(1-\hat{\pi})}{n}}$. We will first save this value as "seTHC" and then we will take a look a the value in the following cell: 

In [ ]:
seTHC <- sqrt(piHatTHC * (1 - piHatTHC) / (36 + 43))

In [ ]:
seTHC

Now for the $z_{\alpha / 2} = z_{.05/2}$, we can use the following:

In [ ]:
qnorm(0.05 / 2, lower.tail = FALSE)

We now have all of the ingredients that we need, let's put them together to find the lower endpoint for the confidence interval: 

In [ ]:
piHatTHC - qnorm(0.05 / 2, lower.tail = FALSE) * seTHC

**Question for you**. Now please find the upper endpoint of this 95% Wald confidence interval. You can copy, paste, and modify code from above into a new code cell below to do this.

As you might expect, there is a function for that. We can use the "BinomCI" function that comes from the *DescTools* package. Since this package has been problematic for us to install, I pulled the functions that we need out of it, and placed it on my website. We can read these functions in using the "source" function. Please run the code below to do this:

In [7]:
source("https://raw.githubusercontent.com/trainorp/iteachAST505/master/Assignments/Labs/functionsForLabs.R")

For the "BinomCI" function the arguments that we need are the number of successes (first argument), the number of trials (second argument), the confidence coefficient ("conf.level" argument), and the method (in our case "wald"): 

In [ ]:
BinomCI(36, 36 + 43, conf.level = 0.95, method = "wald")

**Question for you.** Determine a point estimate and 95% confidence interval for the population proportion of chemotherapy patients for whom Prochlorperaizne would be effective as an antiemetic. It's probably easiest to do this using the BinomCI function again. Please determine these below in a new cell.

Now to determine a 95% WAC confidence interval for the population proportion of chemotherapy patients for whom THC would be effective as an antiemetic. We will first need to find $\tilde{y}$, $\tilde{n}$, $\tilde{\pi}$, and $\text{SE}(\tilde{\pi})$.

For $\tilde{y}$, we have that $\tilde{y}=y + 0.5 \times z_{\alpha / 2}^2$:

In [ ]:
yTildeTHC <- 36 + 0.5 * qnorm(0.05 / 2, lower.tail = FALSE) ^ 2

In [ ]:
yTildeTHC

**Question for you**. Now below, please also find $\tilde{n}$. You may need to consult the lecture notes to do this. Please insert a new code cell and save $\tilde{n}$, as "nTildeTHC". 

Now we can find $\tilde{\pi}$:

In [ ]:
piTildeTHC <- yTildeTHC / nTildeTHC

In [ ]:
piTildeTHC

And now we can find $\text{SE}(\tilde{\pi})$:

In [ ]:
sePiTildeTHC <- sqrt(piTildeTHC * (1 - piTildeTHC) / nTildeTHC)

In [ ]:
sePiTildeTHC

**Question for you.** Now that you have all of the ingredients that you need for a 95% WAC confidence interval, please find the lower and upper endpoints in two separate code cells (please insert below). You should be able to repeat the same process as we used for finding Wald CI's. 

Now we can double check our work using the "BinomCI" function with the 'method = "agresti-coull"' for WAC:

In [ ]:
BinomCI(36, 36 + 43, conf.level = 0.95, method = "agresti-coull")

## 2.2) Hypothesis testing
Now we will discuss hypothesis tests regardin the value of one population parameter. Let's say that we have the following research question, "Is the population proportion of chemotherapy patients for whom THC would be effective as an antiemetic higher than 0.30 or 30%".

**Question for you**. Below in a new markdown cell please state what the null and alternative hypotheses would be for addressing this research question. 

Now, to determine a test statistic, we will use $z^* = \frac{\hat{\pi}-\pi_0}{\text{SE}(\pi_0)}$. So for this we will need to determine $\text{SE}(\pi_0)$. Below, please determine this quantity and save it as "seNullTHC". You can find the code for computing the standard error values above in the lab. 

So then here is our test statistic $z^*$: 

In [ ]:
zStar <- (piHatTHC - .30) / seNullTHC

In [ ]:
zStar

We can now determine the p-value as $P(Z \geq z^*)$: 

In [ ]:
pnorm(zStar, lower.tail = FALSE)

And now here comes the hypothesis test using R: 

In [ ]:
prop.test(x = 36, n = 36 + 43, p = .30, alternative = "greater", correct = FALSE)

# 3) Comparing proportions
Above we determined point estimates and confidence intervals for the population proportion of chemotherapy patients for whom THC would be effective as an antiemetic. Separately we also determined point estimates and confidence intervals for the population proportion of chemotherapy patients for whom Prochlorperazine would be effective as an antiemetic. But we are also interested in if THC or Prochlorperazine is more effective as an antiemetic for chemotherapy patients. So now we need to compare two population proportions. Let's say we want to determine whether THC or Prochlorperazine is more effective by determining point estimates and confidence intervals for $\pi_{\text{THC}} - \pi_{\text{Prochlorperazine}}$.

Let's take a look at this data again:

In [3]:
antiemetics

,Effective,NotEffective
THC,36,43
Prochlorperazine,16,62


Let's use the "BinomDiffCI" function from the infamous *DescTools* package to determine a point estimate and 95% Wald CI for $\pi_{\text{THC}} - \pi_{\text{Prochlorperazine}}$. To do this please go to https://www.rdocumentation.org/packages/DescTools/versions/0.99.34/topics/BinomDiffCI. This shows the help manual entry for this function. Please attempt to use the help manual to guide your efforts to determine the point estimate and CI. Please write the appropriate code below and then add a markdown cell below that to answer the question "is THC or Prochlorperazine is more effective as an antiemetic for chemotherapy patients?"

In [4]:
36 / (36+43) - (16 / (16+62))

[1] 0.250568